In [1]:
#Authors: Amelie Bauerdick
#Wabnitz Lab

# Import Packages

In [2]:
import pandas as pd
import umap
import pandas_bokeh
from sklearn.preprocessing import QuantileTransformer

from bokeh.io import show
from bokeh.models import Div, HoverTool, CustomJS, ColumnDataSource, Slider, Range1d,LinearColorMapper, DataTable, DateFormatter, TableColumn, BooleanFilter, CDSView, CategoricalColorMapper
from bokeh.palettes import RdBu3
from bokeh.io import output_notebook,export_png
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column, row
from bokeh.palettes import all_palettes
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

# Load Data

In [3]:
data=pd.read_csv("path/data_FC_2.CSV")
data

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.016193,0.053811,0.345742,0.054473,...,0.418244,0.115896,0.608014,0.000124,-1.902174,3.904325,G_I_subset,0,1,1
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.031145,0.034854,0.313141,0.067631,...,0.200689,0.070125,0.663636,0.000202,-6.393002,5.862970,G_I_subset,1,1,1
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.043860,0.085238,0.276342,0.083719,...,0.254005,0.058598,0.579506,0.000243,-1.846633,4.701155,G_I_subset,2,1,1
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.034738,0.040915,0.230729,0.036820,...,0.191053,0.067933,0.647519,0.000295,-4.442349,5.586197,G_I_subset,3,1,1
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.025288,0.035094,0.192909,0.051924,...,0.157247,0.070585,0.666216,0.000363,4.843509,6.145188,G_I_subset,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.046299,0.088288,0.181788,0.057115,...,0.200246,0.089658,0.659876,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,3
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.038246,0.052332,0.105424,0.056401,...,0.236768,0.057950,0.640121,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,3
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.028308,0.080975,0.322766,0.048931,...,0.133494,0.048673,0.661283,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,3
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.020407,0.104886,0.236836,0.080854,...,0.268400,0.034229,0.594989,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,3


In [4]:
list(data.columns)

['FSCminusA',
 'FSCminusH',
 'FSCminusW',
 'SSCminusA',
 'SSCminusH',
 'SSCminusW',
 'FJCompminusAPCminusA',
 'FJCompminusAPCminusCy7minusA',
 'FJCompminusBB700minusA',
 'FJCompminusBUV395minusA',
 'FJCompminusBUV496minusA',
 'FJCompminusBUV563minusA',
 'FJCompminusBUV615minusA',
 'FJCompminusBUV661minusA',
 'FJCompminusBUV737minusA',
 'FJCompminusBUV805minusA',
 'FJCompminusBV421minusA',
 'FJCompminusBV510minusA',
 'FJCompminusBV605minusA',
 'FJCompminusBV650minusA',
 'FJCompminusBV711minusA',
 'FJCompminusBV750minusA',
 'FJCompminusBV786minusA',
 'FJCompminusFITCminusA',
 'FJCompminusPEminusA',
 'FJCompminusPEminusCF594minusA',
 'FJCompminusPEminusCy5minusA',
 'FJCompminusPEminusCy7minusA',
 'FJCompminusR718minusA',
 'Time',
 'x',
 'y',
 'file_name',
 'index_file',
 'donor',
 'file_number']

# Scaler for Intensities

In [5]:
def scaler (df,column):
 scaler = QuantileTransformer(output_distribution='uniform')
 scaled_column = scaler.fit_transform(df[[column]])
 df[column] = scaled_column
 return df

In [6]:
scaler(data,'FJCompminusAPCminusA')
scaler(data,'FJCompminusAPCminusCy7minusA')
scaler(data,'FJCompminusBB700minusA') 
scaler(data,'FJCompminusBUV395minusA') 
scaler(data,'FJCompminusBUV496minusA') 
scaler(data,'FJCompminusBUV563minusA')  
scaler(data,'FJCompminusBUV615minusA')     
scaler(data,'FJCompminusBUV661minusA') 
scaler(data,'FJCompminusBUV737minusA') 
scaler(data,'FJCompminusBUV805minusA') 
scaler(data,'FJCompminusBV421minusA') 
scaler(data,'FJCompminusBV510minusA')  
scaler(data,'FJCompminusBV605minusA')  
scaler(data,'FJCompminusBV650minusA')  
scaler(data,'FJCompminusBV711minusA')     
scaler(data,'FJCompminusBV750minusA')     
scaler(data,'FJCompminusBV786minusA')  
scaler(data,'FJCompminusFITCminusA')  
scaler(data,'FJCompminusPEminusA')
scaler(data,'FJCompminusPEminusCF594minusA')
scaler(data,'FJCompminusPEminusCy5minusA') 
scaler(data,'FJCompminusPEminusCy7minusA')
scaler(data,'FJCompminusR718minusA')  

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.070661,0.330543,0.801867,0.277715,...,0.986237,0.953301,0.220687,0.000124,-1.902174,3.904325,G_I_subset,0,1,1
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.536213,0.136720,0.702045,0.528321,...,0.620683,0.550420,0.789400,0.000202,-6.393002,5.862970,G_I_subset,1,1,1
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.824829,0.608138,0.543757,0.750502,...,0.820319,0.303179,0.085013,0.000243,-1.846633,4.701155,G_I_subset,2,1,1
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.641979,0.192596,0.316889,0.050656,...,0.576076,0.503049,0.623173,0.000295,-4.442349,5.586197,G_I_subset,3,1,1
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.332709,0.138815,0.152268,0.230107,...,0.410631,0.559400,0.812096,0.000363,4.843509,6.145188,G_I_subset,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.855453,0.627346,0.117287,0.327560,...,0.618982,0.833492,0.753752,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,3
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.728372,0.314337,0.010226,0.314045,...,0.765454,0.288823,0.535114,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,3
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.440191,0.579469,0.734318,0.180088,...,0.291329,0.128514,0.767508,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,3
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.168860,0.715789,0.347967,0.719043,...,0.856195,0.018794,0.141959,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,3


# DensMAP Conditions

In [10]:
data1=data.copy()

In [11]:
data1['file_number'] = data1['file_number'].astype(str) 
data1['file_number']=data1['file_number'].str.replace(str(1),'G/I')
data1['file_number']=data1['file_number'].str.replace(str(2),'G/I WF10')
data1['file_number']=data1['file_number'].str.replace(str(3),'G/I H\u2082O\u2082')
data1

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.070661,0.330543,0.801867,0.277715,...,0.986237,0.953301,0.220687,0.000124,-1.902174,3.904325,G_I_subset,0,1,G/I
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.536213,0.136720,0.702045,0.528321,...,0.620683,0.550420,0.789400,0.000202,-6.393002,5.862970,G_I_subset,1,1,G/I
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.824829,0.608138,0.543757,0.750502,...,0.820319,0.303179,0.085013,0.000243,-1.846633,4.701155,G_I_subset,2,1,G/I
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.641979,0.192596,0.316889,0.050656,...,0.576076,0.503049,0.623173,0.000295,-4.442349,5.586197,G_I_subset,3,1,G/I
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.332709,0.138815,0.152268,0.230107,...,0.410631,0.559400,0.812096,0.000363,4.843509,6.145188,G_I_subset,4,1,G/I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.855453,0.627346,0.117287,0.327560,...,0.618982,0.833492,0.753752,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,G/I H₂O₂
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.728372,0.314337,0.010226,0.314045,...,0.765454,0.288823,0.535114,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,G/I H₂O₂
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.440191,0.579469,0.734318,0.180088,...,0.291329,0.128514,0.767508,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,G/I H₂O₂
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.168860,0.715789,0.347967,0.719043,...,0.856195,0.018794,0.141959,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,G/I H₂O₂


In [ ]:
output_file('html/DensMAP.html')

source=ColumnDataSource(data1)

cm = CategoricalColorMapper( palette=['purple','dodgerblue','red'],            
                             factors=['G/I','G/I WF10', 'G/I H\u2082O\u2082']
                           )    

hover_emb = HoverTool(names=["data1"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">group:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )

tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  

dmap = figure(plot_width=600,
                  plot_height=600, 
                  tools=tools_emb,
                  x_axis_label='DensMAP 1',
                  y_axis_label='DensMAP 2',  
                  title='DensMAP'
             )

dmap.circle('x', 'y',
                size=4, 
                fill_color={'field': 'file_number', 'transform': cm},
                alpha=0.7, 
                line_alpha=0, 
                line_width=0.03, 
                source=source, 
                name="data1",
                legend_group='file_number'
              )
dmap.x_range = Range1d(-10, 18)
dmap.y_range = Range1d(-5, 20)

dmap.outline_line_color= 'white' 
dmap.xaxis.visible = True
dmap.yaxis.visible = True
dmap.grid.visible = False
dmap.xgrid.visible = False
dmap.ygrid.visible = False

dmap.xaxis.axis_label_text_font_size = "20pt"
dmap.yaxis.axis_label_text_font_size = "20pt"
dmap.xaxis.major_label_text_font_size = "20pt"
dmap.yaxis.major_label_text_font_size = "20pt"

dmap.title.text_font_size = '30pt'
dmap.add_layout(dmap.legend[0], 'center')
dmap.legend.title_text_font_style = "bold"
dmap.legend.background_fill_alpha = 0.0
dmap.legend.border_line_alpha=0
dmap.legend.label_text_font_size = '20pt'
dmap.legend.title_text_font_size = '20pt'
dmap.legend.glyph_height = 30
dmap.legend.glyph_width = 30

show(dmap)

# DensMAP G/I

In [25]:
data1=data.copy()
data1['file_number'] = (data1['file_number']==1).astype(int)
data1

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.070661,0.330543,0.801867,0.277715,...,0.986237,0.953301,0.220687,0.000124,-1.902174,3.904325,G_I_subset,0,1,1
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.536213,0.136720,0.702045,0.528321,...,0.620683,0.550420,0.789400,0.000202,-6.393002,5.862970,G_I_subset,1,1,1
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.824829,0.608138,0.543757,0.750502,...,0.820319,0.303179,0.085013,0.000243,-1.846633,4.701155,G_I_subset,2,1,1
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.641979,0.192596,0.316889,0.050656,...,0.576076,0.503049,0.623173,0.000295,-4.442349,5.586197,G_I_subset,3,1,1
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.332709,0.138815,0.152268,0.230107,...,0.410631,0.559400,0.812096,0.000363,4.843509,6.145188,G_I_subset,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.855453,0.627346,0.117287,0.327560,...,0.618982,0.833492,0.753752,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,0
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.728372,0.314337,0.010226,0.314045,...,0.765454,0.288823,0.535114,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,0
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.440191,0.579469,0.734318,0.180088,...,0.291329,0.128514,0.767508,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,0
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.168860,0.715789,0.347967,0.719043,...,0.856195,0.018794,0.141959,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,0


In [ ]:
output_file('html/DensMAP_Control.html')

source=ColumnDataSource(data1)
  
cm = LinearColorMapper( palette=['#00000000','purple'],                      
                            low = min(data1['file_number']),
                           high = max(data1['file_number']) 
                      )    

hover_emb = HoverTool(names=["data1"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file number:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )


tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  

dmap = figure(plot_width=600,
              plot_height=600, 
              tools=tools_emb,
              x_axis_label='DensMAP 1',
              y_axis_label='DensMAP 2',
              title="G/I"
             )

dmap.circle('x', 'y',
            size=4, 
            fill_color={'field': 'file_number', 'transform': cm},
            alpha=0.7, 
            line_alpha=0, 
            line_width=0.03, 
            source=source, 
            name="data1",
            legend_group='file_number'
         )

dmap.x_range = Range1d(-10, 15)
dmap.y_range = Range1d(-5, 18)

dmap.title.align = 'center'
dmap.grid.visible = False
dmap.axis.visible = False
dmap.outline_line_color= None
dmap.border_fill_color = None

dmap.legend.visible=False 
dmap.title.text_font_size = '60pt'

show(dmap)

# DensMAP G/I WF10

In [37]:
#Überprüfen
data1=data.copy()
data1['file_number'] = (data1['file_number']==2).astype(int)
data1

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.070661,0.330543,0.801867,0.277715,...,0.986237,0.953301,0.220687,0.000124,-1.902174,3.904325,G_I_subset,0,1,0
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.536213,0.136720,0.702045,0.528321,...,0.620683,0.550420,0.789400,0.000202,-6.393002,5.862970,G_I_subset,1,1,0
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.824829,0.608138,0.543757,0.750502,...,0.820319,0.303179,0.085013,0.000243,-1.846633,4.701155,G_I_subset,2,1,0
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.641979,0.192596,0.316889,0.050656,...,0.576076,0.503049,0.623173,0.000295,-4.442349,5.586197,G_I_subset,3,1,0
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.332709,0.138815,0.152268,0.230107,...,0.410631,0.559400,0.812096,0.000363,4.843509,6.145188,G_I_subset,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.855453,0.627346,0.117287,0.327560,...,0.618982,0.833492,0.753752,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,0
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.728372,0.314337,0.010226,0.314045,...,0.765454,0.288823,0.535114,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,0
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.440191,0.579469,0.734318,0.180088,...,0.291329,0.128514,0.767508,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,0
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.168860,0.715789,0.347967,0.719043,...,0.856195,0.018794,0.141959,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,0


In [ ]:
output_file('html/DensMAP_WF10.html')

source=ColumnDataSource(data1)

cm = LinearColorMapper( palette=['#00000000','dodgerblue'],                    
                            low = min(data1['file_number']),
                           high = max(data1['file_number'])
                      )    

hover_emb = HoverTool(names=["data1"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file number:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )


tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  
    
dmap = figure(plot_width=600,
              plot_height=600, 
              tools=tools_emb,
              x_axis_label='DensMAP 1',
              y_axis_label='DensMAP 2',
              title="G/I WF10"
             )

dmap.circle('x', 'y',
            size=4, 
            fill_color={'field': 'file_number', 'transform': cm},
            alpha=0.7, 
            line_alpha=0, 
            line_width=0.03, 
            source=source, 
            name="data1",
            legend_group='file_number'
            )

dmap.x_range = Range1d(-10, 15)
dmap.y_range = Range1d(-5, 18)

dmap.title.align = 'center'
dmap.grid.visible = False
dmap.axis.visible = False
dmap.outline_line_color= None
dmap.border_fill_color = None

dmap.legend.visible=False 
dmap.title.text_font_size = '60pt'

show(dmap)

# DensMAP G/I H2O2

In [29]:
data1=data.copy()
data1['file_number'] = (data1['file_number']==3).astype(int)
data1

,FSCminusA,FSCminusH,FSCminusW,SSCminusA,SSCminusH,SSCminusW,FJCompminusAPCminusA,FJCompminusAPCminusCy7minusA,FJCompminusBB700minusA,FJCompminusBUV395minusA,...,FJCompminusPEminusCy5minusA,FJCompminusPEminusCy7minusA,FJCompminusR718minusA,Time,x,y,file_name,index_file,donor,file_number
0,0.243210,0.363422,0.328837,0.482799,0.645013,0.350173,0.070661,0.330543,0.801867,0.277715,...,0.986237,0.953301,0.220687,0.000124,-1.902174,3.904325,G_I_subset,0,1,0
1,0.213119,0.349254,0.271121,0.318340,0.457919,0.273632,0.536213,0.136720,0.702045,0.528321,...,0.620683,0.550420,0.789400,0.000202,-6.393002,5.862970,G_I_subset,1,1,0
2,0.262130,0.407060,0.302377,0.310960,0.432206,0.292383,0.824829,0.608138,0.543757,0.750502,...,0.820319,0.303179,0.085013,0.000243,-1.846633,4.701155,G_I_subset,2,1,0
3,0.209967,0.320992,0.314092,0.343209,0.473436,0.327114,0.641979,0.192596,0.316889,0.050656,...,0.576076,0.503049,0.623173,0.000295,-4.442349,5.586197,G_I_subset,3,1,0
4,0.174743,0.256952,0.295978,0.521453,0.676782,0.366800,0.332709,0.138815,0.152268,0.230107,...,0.410631,0.559400,0.812096,0.000363,4.843509,6.145188,G_I_subset,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38995,0.280539,0.422851,0.338473,0.317581,0.455609,0.281744,0.855453,0.627346,0.117287,0.327560,...,0.618982,0.833492,0.753752,0.598153,-4.761563,10.836417,G_I_H2O2_subset,13652,1,1
38996,0.220835,0.307916,0.360680,0.457540,0.623184,0.346606,0.728372,0.314337,0.010226,0.314045,...,0.765454,0.288823,0.535114,0.598195,-4.952386,11.097390,G_I_H2O2_subset,13653,1,1
38997,0.246989,0.363781,0.314847,0.256674,0.368279,0.271887,0.440191,0.579469,0.734318,0.180088,...,0.291329,0.128514,0.767508,0.598289,-3.696257,10.620928,G_I_H2O2_subset,13654,1,1
38998,0.275247,0.417453,0.316368,0.441002,0.612630,0.322775,0.168860,0.715789,0.347967,0.719043,...,0.856195,0.018794,0.141959,0.598290,-2.990655,10.731456,G_I_H2O2_subset,13655,1,1


In [ ]:
output_file('html/DensMAP_H2O2.html')

source=ColumnDataSource(data1)

cm = LinearColorMapper( palette=['#00000000','red'],                    
                            low = min(data1['file_number']),
                           high = max(data1['file_number']) 
                      )    

hover_emb = HoverTool(names=["data1"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file number:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )

tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  
    
dmap = figure(plot_width=600,
              plot_height=600, 
              tools=tools_emb,
              x_axis_label='DensMAP 1',
              y_axis_label='DensMAP 2',
              title='G/I H\u2082O\u2082'
             )

dmap.circle('x', 'y',
            size=4, 
            fill_color={'field': 'file_number', 'transform': cm},
            alpha=0.7, 
            line_alpha=0, 
            line_width=0.03, 
            source=source, 
            name="data1",
            legend_group='file_number'
            )

dmap.x_range = Range1d(-10, 15)
dmap.y_range = Range1d(-5, 18)

dmap.title.align = 'center'
dmap.grid.visible = False
dmap.axis.visible = False
dmap.outline_line_color= None
dmap.border_fill_color = None

dmap.legend.visible=False 
dmap.title.text_font_size = '60pt'

show(dmap)

# DensMAP Donor

In [ ]:
output_file('html/DensMAP_Donor.html')

source=ColumnDataSource(data)
  
cm = LinearColorMapper( palette=['red','blue','darkgray'],            
                        low = min(data['donor']),
                        high = max(data['donor']) 
                      )    

hover_emb = HoverTool(names=["data"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file number:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )

tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  
    
dmap = figure(plot_width=600,
              plot_height=600, 
              tools=tools_emb,
              x_axis_label='DensMAP 1',
              y_axis_label='DensMAP 2',
              title="Donors")

dmap.circle('x', 'y',
            size=4, 
            fill_color={'field': 'donor', 'transform': cm},
            alpha=0.7, 
            line_alpha=0, 
            line_width=0.03, 
            source=source, 
            name="data",
            legend_group='donor'
            )

dmap.x_range = Range1d(-10, 15)
dmap.y_range = Range1d(-5, 18)

dmap.outline_line_color= 'white' 
dmap.xaxis.visible = True
dmap.yaxis.visible = True
dmap.grid.visible = False
dmap.xgrid.visible = False
dmap.ygrid.visible = False

dmap.xaxis.axis_label_text_font_size = "20pt"
dmap.yaxis.axis_label_text_font_size = "20pt"
dmap.xaxis.major_label_text_font_size = "20pt"
dmap.yaxis.major_label_text_font_size = "20pt"
dmap.add_layout(dmap.legend[0], 'center')
dmap.legend.title_text_font_style = "bold"
dmap.legend.background_fill_alpha = 0.0
dmap.legend.border_line_alpha=1.0
dmap.legend.label_text_font_size = '15pt'
dmap.legend.title_text_font_size = '20pt'
dmap.title.text_font_size = '30pt'

show(dmap)

# DensMAP stain

In [ ]:
def dmap(feature,title,output):
    
 output_file(output)

 source=ColumnDataSource(data)
  
 cm = LinearColorMapper( palette='Plasma256',            
                        low = min(data[feature]),
                        high = max(data[feature]) 
                      )    


 hover_emb = HoverTool(names=["data"], 
                      tooltips="""
                                 <div style="margin: 10">
                                  <div style="margin: 0 auto; width:300px;">
                                    <span style="font-size: 12px; font-weight: bold;">index:</span>
                                     <span style="font-size: 12px">@index <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">index file:</span>
                                     <span style="font-size: 12px">@index_file <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file number:</span>
                                     <span style="font-size: 12px">@file_number <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">donor:</span>
                                     <span style="font-size: 12px">@donor <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">file name:</span>
                                     <span style="font-size: 12px">@file_name <br /> </span>
                                    <span style="font-size: 12px; font-weight: bold;">cluster:</span>
                                     <span style="font-size: 12px">@cluster <br /> </span>
                                 </div>
                                </div>
                               """
                     )


 tools_emb = ['save','lasso_select','pan', 'wheel_zoom', 'reset',hover_emb]  
    
 dmap = figure(plot_width=500,
              plot_height=500, 
              tools=tools_emb,
              x_axis_label='DensMAP 1',
              y_axis_label='DensMAP 2',
              title=title)

 dmap.circle('x', 'y',
            size=4, 
            fill_color={'field': feature, 'transform': cm},
            alpha=0.7, 
            line_alpha=0, 
            line_width=0.03, 
            source=source, 
            name="data",
            )

 dmap.x_range = Range1d(-10, 15)
 dmap.y_range = Range1d(-5, 18)

 dmap.title.align = 'center'
 dmap.grid.visible = False
 dmap.axis.visible = False
 dmap.outline_line_color= None
 dmap.border_fill_color = None

 dmap.title.text_font_size = '33pt'

 show(dmap)

In [ ]:
dmap ('FJCompminusAPCminusCy7minusA','CD66b','html/DensMAP_CD66b.html')
dmap ('FJCompminusBB700minusA','Siglec-8','html/DensMAP_Siglec8.html')
dmap ('FJCompminusBUV395minusA','CD11b','html/DensMAP_CD11b.html')
dmap ('FJCompminusBUV496minusA','CD14','html/DensMAP_CD14.html')

dmap ('FJCompminusBUV563minusA','CD10','html/DensMAP_CD10.html')
dmap ('FJCompminusBUV615minusA','CD36','html/DensMAP_CD36.html')
dmap ('FJCompminusBUV661minusA','CD33','html/DensMAP_CD33.html')
dmap ('FJCompminusBUV737minusA','HLA-DR','html/DensMAP_HLA_DR.html')
dmap ('FJCompminusBUV805minusA','CD16','html/DensMAP_CD16.html')
dmap ('FJCompminusBV421minusA','CXCR4','html/DensMAP_CXCR4.html')
dmap ('FJCompminusBV510minusA','CD64','html/DensMAP_CD64.html')
dmap ('FJCompminusBV605minusA','CD177','html/DensMAP_CD177.html')
dmap ('FJCompminusBV650minusA','PD-L1','html/DensMAP_PDL1.html')
dmap ('FJCompminusBV711minusA','CD11c','html/DensMAP_CD11c.html')
dmap ('FJCompminusBV750minusA','CD80','html/DensMAP_CD80.html')
dmap ('FJCompminusBV786minusA','CD62L','html/DensMAP_CD62L.html')
dmap ('FJCompminusFITCminusA','Apotracker','html/DensMAP_Apotracker.html')
dmap ('FJCompminusPEminusA','CXCR2','html/DensMAP_CXCR2.html')
dmap ('FJCompminusPEminusCF594minusA','CCR5','html/DensMAP_CCR5.html')
dmap ('FJCompminusPEminusCy5minusA','CD95','html/DensMAP_CD95.html')
dmap ('FJCompminusPEminusCy7minusA','TNFR2','html/DensMAP_TNFR2.html')
dmap ('FJCompminusR718minusA','MPO','html/DensMAP_MPO.html')
